# Derivative of traits_i with respect to traits_i

Here I will be testing my solution for
$\frac{ \partial \mathbf{\hat{V}_i} }{ \partial \mathbf{V_i} }$
(see below)
by calculating the Jacobian using the `theano` package
and comparing those results to my solution.




## Importing packages and setting options

In [19]:
%env OMP_NUM_THREADS=4
%env THEANO_FLAGS='openmp=True'
import sympy
import theano
theano.config.cxx = ""
import theano.tensor as T
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
pd.options.display.max_columns = 10

env: OMP_NUM_THREADS=4
env: THEANO_FLAGS='openmp=True'


## Equations

__Notes:__

- ${}^\text{T}$ represents transpose.
- Elements in __bold__ are matrices
- Multiplication between matrices is always matrix multiplication, not
  element-wise
  

The equations for (1) traits for species $i$ at time $t+1$ ($\mathbf{V}_{i,t+1}$)
and (2) the partial derivative of species $i$ traits at time $t+1$ with respect
to species $i$ traits at time  $t$ are as follows:

\begin{align}
\mathbf{V}_{i,t+1} &= \mathbf{V}_{i,t} + 2 ~ \sigma_i^2
    \left[
        \alpha_0 ~ \mathbf{\Omega}_{i,t} ~
            \textrm{e}^{-\mathbf{V}_{i,t}^\textrm{T} \mathbf{V}_{i,t}} ~ \mathbf{V}_{i,t}^\textrm{T}
        - f ~ \mathbf{V}_{i,t}^\textrm{T} \mathbf{C}
    \right] \\
    \frac{ \partial \mathbf{V}_{i,t+1} }{ \partial \mathbf{V}_{i,t} } &= \mathbf{I} + 2 ~ \sigma_i^2 ~
        \left[
            \alpha_0 ~ \mathbf{\Omega}_{i,t} ~ \textrm{e}^{ - \mathbf{V}_{i,t}^{\textrm{T}} \mathbf{V}_{i,t} }
            \left(
                \mathbf{I} - 2 ~ \mathbf{V}_{i,t} \mathbf{V}_{i,t}^{\textrm{T}}
            \right) -
            f \: \mathbf{C}^{\textrm{T}}
        \right] \\
    \mathbf{\Omega}_{i,t} &\equiv N_{i,t} +
            \sum_{j \ne i}^{n}{ N_{j,t} \; \textrm{e}^{ - \mathbf{V}_{j,t}^\textrm{T} \mathbf{D} \mathbf{V}_{j,t} } }
\end{align}


## Read CSV of simulated datasets

In [20]:
sims = pd.read_csv("simulated_data.csv")
sims.head()

,V1,V2,V3,V4,V5,...,f,a0,eta,r0,d
0,4.945110,2.869199,6.747126,6.142522,5.629532,...,0.068890,0.112113,-0.331150,1.422746,-0.091228
1,0.718846,1.220364,0.815571,0.868633,0.838021,...,0.309021,0.057579,0.094811,1.237047,0.003429
2,3.369285,1.912974,3.131174,0.046303,1.416252,...,0.118318,0.401410,-0.036977,1.746024,0.012160
3,0.373669,0.283873,0.237735,0.053632,0.062281,...,0.497286,0.499730,0.117188,0.669199,0.081612
4,3.562637,1.635016,5.724176,4.953962,1.060083,...,0.042638,0.307171,-0.467453,0.952351,0.051834


## Functions to compare methods

In [21]:
def automatic(i, N, V, O, C, f, a0, s2):
    """Automatic differentiation using theano pkg"""
    Vi = T.dvector('Vi')
    Vhat = Vi + 2 * s2 * (
        ( a0 * O * T.exp(-1 * T.dot(Vi.T, Vi)) * Vi.T) - 
        ( f * T.dot(Vi.T, C) )
    )
    J, updates = theano.scan(lambda i, Vhat, Vi : T.grad(Vhat[i], Vi), 
                         sequences=T.arange(Vhat.shape[0]), non_sequences=[Vhat, Vi])
    num_fun = theano.function([Vi], J, updates=updates)
    out_array = num_fun(V[:,i])
    return out_array

In [25]:
def symbolic(i, V, O, C, f, a0, s2):
    """Symbolic differentiation using math"""
    q = V.shape[0]
    I = np.identity(q)
    Vi = V[:,i]
    Vi = Vi.reshape((q, 1))
    dVhat = I + 2 * s2 * (
        ( a0 * O * np.exp(-1 * Vi.T @ Vi)[0,0] * (I - 2 * Vi @ Vi.T) ) -
        (f * C.T)
    )
    return dVhat

In [26]:
def compare_methods(sim_i, s2 = 0.01, abs = False):
    """Compare answers from symbolic and automatic methods"""
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    n, q = (N.size, int(V.size / N.size))
    V = V.reshape((q, n), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, d)
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)
    
    # Create output array:
    diffs = np.empty((n, 4))
    diffs[:,0] = sim_i
    
    # Fill output array:
    for i in range(0, n):
        O = N[i] + np.sum([np.exp(-1 * np.dot(np.dot(V[:,j].T, D), V[:,j])) * N[j] 
            for j in range(0, N.size) if j != i])
        auto = automatic(i, N, V, O, C, f, a0, s2)
        sym = symbolic(i, V, O, C, f, a0, s2)
        if abs:
            diff = auto - sym
        else:
            diff = (auto - sym) / sym
        diff = diff.flatten()
        diffs[i, 1] = i
        diffs[i, 2] = diff.min()
        diffs[i, 3] = diff.max()
    
    return diffs

### Example of using `compare_methods`:

In [28]:
diffs = compare_methods(0)
# Worst case examples:
print(diffs[:,2].min())
print(diffs[:,3].max())

0.0
1.242113716418726e-16


## Comparing methods

This takes ~2 minutes.

In [29]:
n_per_rep = 4
diffs = np.empty((int(n_per_rep * 100), 4))

In [30]:
for rep in tqdm(range(100)):
    diffs_r = compare_methods(rep)
    diffs[(rep * n_per_rep):((rep+1) * n_per_rep),:] = diffs_r

 43%|████▎     | 43/100 [01:56<02:46,  2.92s/it]INFO (theano.gof.compilelock): Waiting for existing lock by process '51286' (I am process '50793')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/lucasnell/.theano/compiledir_Darwin-19.4.0-x86_64-i386-64bit-i386-3.7.7-64/lock_dir
100%|██████████| 100/100 [05:09<00:00,  3.09s/it]


## The results
They appear to be extremely similar, enough so that I feel comfortable with my symbolic version.

In [34]:
print(diffs[:,2].min())
print(diffs[:,3].max())

-3.607093518239362e-14
8.778068698894372e-15


## Write output to file

To make sure the R version works, too, I'm writing to a CSV file the output from the symbolic version on the 100 datasets.

In [35]:
n = np.sum([x.startswith("N") for x in sims.columns])
q = int(np.sum([x.startswith("V") for x in sims.columns]) / n)
s2 = 0.01
# Output array
results = np.zeros((100, n * q * q))

for sim_i in range(100):
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    V = V.reshape((q, n), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)

    # Fill output array:
    for i in range(0, n):
        O = N[i] + np.sum([np.exp(-d * np.dot(V[:,j].T, V[:,j])) * N[j] 
            for j in range(0, N.size) if j != i])
        sym = symbolic(i, V, O, C, f, a0, s2)
        results[sim_i, (i*q*q):((i+1)*q*q)] = sym.flatten()

# Make sure first and last aren't zeros:
results[[0, 99], :]

array([[ 9.98622203e-01,  4.56257198e-04,  4.56257198e-04,
         4.56257198e-04,  9.98622203e-01,  4.56257198e-04,
         4.56257198e-04,  4.56257198e-04,  9.98622203e-01,
         9.98622203e-01,  4.56257198e-04,  4.56257198e-04,
         4.56257198e-04,  9.98622203e-01,  4.56257198e-04,
         4.56257198e-04,  4.56257198e-04,  9.98622203e-01,
         9.98622203e-01,  4.56257198e-04,  4.56257198e-04,
         4.56257198e-04,  9.98622203e-01,  4.56257198e-04,
         4.56257198e-04,  4.56257198e-04,  9.98622203e-01,
         9.98613046e-01,  4.40873097e-04,  4.42520935e-04,
         4.40873097e-04,  9.98601505e-01,  4.36434345e-04,
         4.42520935e-04,  4.36434345e-04,  9.98606007e-01],
       [ 9.31362535e-01, -2.61607428e-02, -1.88111405e-02,
        -2.61607428e-02,  1.00223649e+00, -5.87081473e-03,
        -1.88111405e-02, -5.87081473e-03,  1.00649798e+00,
         9.96889722e-01,  6.74698562e-04,  6.74721155e-04,
         6.74698562e-04,  9.96889732e-01,  6.74722367e-

In [36]:
np.savetxt('results/dVi_dVi.csv', results, delimiter=',')